## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [19]:
import pandas as pd
import numpy as np

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
sms_spam_tbl = pd.read_csv('/content/drive/MyDrive/data/sms_spam.csv')

In [37]:
sms_spam_tbl.head(10)

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
5,ham,Aiya we discuss later lar... Pick u up at 4 is...
6,ham,Are you this much buzy
7,ham,Please ask mummy to call father
8,spam,Marvel Mobile Play the official Ultimate Spide...
9,ham,"fyi I'm at usf now, swing by the room whenever"


In [38]:
sms_spam_tbl.shape

(5559, 2)

In [39]:
sms_spam_tbl['numType'] = sms_spam_tbl['type'].map({'ham':0, 'spam':1})
sms_spam_tbl.drop(columns=['type'], inplace=True)

In [40]:
sms_spam_tbl['count']=0
for i in np.arange(0,len(sms_spam_tbl.text)):
    sms_spam_tbl.loc[i,'count'] = len(sms_spam_tbl.loc[i,'text'])

In [31]:
import nltk
from nltk.tokenize import word_tokenize          
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
wnl = WordNetLemmatizer()
sms_spam_lem = sms_spam_tbl.copy()
sms_spam_lem['text'] = sms_spam_tbl['text'].apply(lambda x: ' '.join(wnl.lemmatize(t.lower()) for t in word_tokenize(x)))


In [42]:
sms_spam_lem.head()

,type,text,numType,count
0,ham,hope you are having a good week . just checkin...,0,49
1,ham,k..give back my thanks .,0,23
2,ham,am also doing in cbe only . but have to pay .,0,43
3,spam,"complimentary 4 star ibiza holiday or £10,000 ...",1,149
4,spam,okmail : dear dave this is your final notice t...,1,161


In [ ]:
# different vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF
tfidf_model = TfidfVectorizer()
sms_spam_tfidf = sms_spam_lem.copy()
sms_spam_lem['text']=tfidf_model.fit_transform(sms_spam_lem['text'])


In [ ]:
# Count



In [ ]:
# Count(analyzer='char_wb')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
lgbmodel_bst = lgb.LGBMClassifier(max_depth=6, n_estimators=200, num_leaves=40)
param_grid = {
    'num_leaves': list(range(8, 92, 4)),
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'bagging_freq': [3, 4, 5, 6, 7],
    'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10),
    "min_split_gain": [0.0, 0.1, 0.01],
    "min_child_weight": [0.001, 0.01, 0.1, 0.001],
    "min_child_samples": [20, 30, 25],
    "subsample": [1.0, 0.5, 0.8],
}
model = RandomizedSearchCV(lgbmodel_bst, param_grid, random_state=42)
search = model.fit(X_train, y_train)
search.best_params_

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [2]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [16]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

 Оно пересекается с 'abca'


**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [29]:
text = "AV is largest Analytics community of India"
result = re.findall(r'\b\w{1,2}', text) 
print(result)

['AV', 'is', 'la', 'An', 'I', 'co', 'of', 'In']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [32]:
result = re.split(r'\.', 'itsy. bitsy. teenie. weenie.', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie. weenie.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [35]:
result = re.sub(r'\d', 'DIG', '132')
print(result)

DIGDIGDIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [7]:
result = re.sub(r'\b\S+://\S+\b', '', 'URL could have the form http://www.example.com/index.html, which indicates a protocol ( http )')
print(result)

URL could have the form , which indicates a protocol ( http )


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [9]:
prog = re.compile(r'[А-Яа-яё\-]{3,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [10]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
prog = re.compile(r'@\S+\b')
prog.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']